In [1]:
import os
import time
import nbimporter

In [2]:
import utils
import blockchain as blockchain_utils

Importing Jupyter notebook from utils.ipynb
Importing Jupyter notebook from blockchain.ipynb


### LOAD RESOURCES

In [3]:
settings = utils.load_yaml('resources/settings.yaml')

In [4]:
identifier = utils.load_yaml('resources/identifier.yaml')

In [5]:
latest = utils.load_json('resources/latest.json')

### CONNECT TO THE ETHEREUM GATEWAY

In [6]:
web3 = blockchain_utils.connect(settings)

### SERIALIZE THE MANAGER CONTRACT

In [7]:
device_manager = blockchain_utils.contract(latest['devicemanager'], web3, settings)

### HASH THE DEVICE IDENTIFIER

In [8]:
hashed_identifier = utils.hash_data(identifier)

### ATTEMPT TO FETCH A CONTRACT WITH THE DEVICE HASH

In [9]:
device_address = device_manager.read({
    'func': 'fetch_device',
    'params': hashed_identifier
})

In [10]:
# IF IT DOES NOT EXIST
if device_address == '0x0000000000000000000000000000000000000000':
    
    # REGISTER IT
    device_manager.write({
        'func': 'add',
        'params': [hashed_identifier]
    })
    
    # FETCH THE ADDRESS AGAIN
    device_address = device_manager.read({
        'func': 'fetch_device',
        'params': hashed_identifier
    })

### SERIALIZE THE DEVICE SPECIFIC CONTRACT

In [11]:
device_contract = blockchain_utils.contract({
    'address': device_address,
    'abi': latest['device']['abi']
}, web3, settings)

### CALLBACK FUNC FOR UPLOADING DATA

In [12]:
def track(file_path):
    
    # OPEN THE FILE
    file = open(file_path, 'r')

    # FIND THE TAILEND OF THE FILE
    file_stats = os.stat(file_path)
    file_size = file_stats[6]
    
    # GOTO THE LAST ROW
    file.seek(file_size)
    
    # LINE CONTAINER
    container = []

    try:
        print('READING LOG...\n')
        
        # EVENT LOOP
        while True:

                # READ THE LAST LINE
                where = file.tell()
                line = file.readline()

                # IF THE LINE ISNT EMPTY
                if line:

                    # PARSE & ENCODE THE LINE
                    parsed = utils.parse_line(line)
                    encoded = utils.encode(parsed)

                    # APPEND IT TO THE CONTAINER
                    container.append(encoded)

                # OTHERWISE, SLEEP FOR A SECOND TO PREVENT SPAM
                else:
                    time.sleep(1)

                # IF THE TAILEND OF THE FILE WAS REACHED
                # AND CONTAINER IS NOT EMPTY
                if file.tell() == os.path.getsize(file_path) and len(container) > 0:

                    # PUSH THE HASHED DATA
                    device_contract.write({
                        'func': 'add',
                        'params': [container]
                    })

                    print('UPLOADED ', len(container), 'LINES')

                    # RESET THE CONTAINER
                    container = []

    # MANUALLY KILL THE PROCESS
    except:
        print('\nTHE PROCESS WAS MANUALLY KILLED')

### START TRACKING LOG FILE

In [13]:
track(settings['log_path'])

READING LOG...


THE PROCESS WAS MANUALLY KILLED
